# LlamaParse JSON Mode + Multimodal RAG

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/parse/demo_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to use LlamaParse JSON mode with LlamaIndex to build a simple multimodal RAG pipeline.

Using JSON mode gives you back a list of json dictionaries, which contains both text and images. You can then download these images and use a multimodal model to extract information and index them.

## Setup

Define imports, env variables, global LLM/embedding models.

In [ ]:
%pip install llama-index
%pip install llama-index-core
%pip install llama-index-llms-anthropic
%pip install llama-index-embeddings-huggingface
%pip install llama-cloud-services

In [ ]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

# Using Anthropic API for embeddings/LLMs
os.environ["ANTHROPIC_API_KEY"] = "sk-"

In [ ]:
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model="claude-3-5-sonnet-20241022")

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

## Load Data

Let's load in the Uber 10Q report.

In [ ]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O './uber_10q_march_2022.pdf'

## Using LlamaParse in JSON Mode for PDF Reading

We show you how to run LlamaParse in JSON mode for PDF reading.

In [ ]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(take_screenshot=True)
result = await parser.aparse("./uber_10q_march_2022.pdf")

Started parsing the file under job_id cf5a4f51-1af8-47f7-9b3d-80a905d06b89


In [ ]:
text_nodes = await result.aget_text_nodes(split_by_page=True)
image_nodes = await result.aget_image_nodes(
    include_screenshot_images=True,
    include_object_images=True,
    image_download_dir="./uber_10q_images",
)

## Extract/Index images from image dicts

Here we use a multimodal model to caption images and create text nodes for indexing.

In [ ]:
!mkdir -p llama2_images

from llama_index.core.llms import ChatMessage, ImageBlock, TextBlock
from llama_index.core.schema import ImageNode, TextNode
from llama_index.llms.anthropic import Anthropic


def get_image_text_nodes(image_nodes: list[ImageNode]):
    """Extract out text from images using a multimodal model."""
    llm = Anthropic(model="claude-3-5-haiku-20241022", max_tokens=300)
    img_text_nodes = []
    for image_node in image_nodes:
        image_path = image_node.image_path
        message = ChatMessage(
            role="user",
            blocks=[
                TextBlock(text="Describe the images as alt text"),
                ImageBlock(path=image_path),
            ],
        )
        response = llm.chat([message])
        text_node = TextNode(
            text=str(response.message.content), metadata={"path": image_path}
        )
        img_text_nodes.append(text_node)

    return img_text_nodes

mkdir: llama2_images: File exists


In [ ]:
image_text_nodes = get_image_text_nodes(image_nodes)

In [ ]:
image_text_nodes[0].get_content()

'The image shows a bar graph titled "Monthly Active Platform Consumers (in millions)". The graph displays data from Q2 2020 to Q1 2022 over 8 quarters. The number of monthly active platform consumers starts at 55 million in Q2 2020 and steadily increases each quarter, reaching 115 million by Q1 2022. The graph illustrates consistent quarter-over-quarter growth in this metric over the nearly 2 year time period shown.'

## Build Index across image and text nodes

Here we build a vector index across both text nodes and text nodes extracted from images.

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(text_nodes + image_text_nodes)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
# ask question over image!
response = query_engine.query(
    "What does the bar graph titled 'Monthly Active Platform Consumers' show?"
)
print(str(response))

The bar graph titled "Monthly Active Platform Consumers (in millions)" shows the number of monthly active consumers on Uber's platform over a period of 8 quarters from Q2 2020 to Q1 2022. 

The graph indicates steady quarter-over-quarter growth in this metric, starting at 55 million monthly active platform consumers in Q2 2020 and increasing each quarter to reach 115 million by Q1 2022. This represents consistent growth in Uber's user base on their platform over the nearly 2 year period shown in the graph.


In [ ]:
# ask question over text!
response = query_engine.query("What are the main risk factors for Uber?")
print(str(response))

Based on the context provided, some of the main risk factors for Uber include:

- A significant percentage of Uber's bookings come from large metropolitan areas, which could be negatively impacted by various economic, social, weather, regulatory and other conditions, including COVID-19.

- Uber may fail to successfully offer autonomous vehicle technologies on its platform or these technologies may not perform as expected. 

- Retaining and attracting high-quality personnel is important for Uber's business and continued attrition could adversely impact the company.

- Security breaches, data privacy issues, cyberattacks and unauthorized access to Uber's proprietary data and systems pose risks.

- Uber is subject to climate change risks, both physical and transitional, that could adversely impact its business if not managed properly. 

- Uber relies on third parties for open marketplaces to distribute its platform and software, and interference from these third parties could harm its bus